# **RoNIN Dataset Preprocessing Utility**
### **Batch Processing: HDF5 to Standardized CSV Format**

**Author:** Ilya Afanasyev  
**Project:** Manifold-based Navigation Framework: Filters & Observers 

### **Overview**

This notebook automates the extraction and standardization of the **SFU RoNIN Dataset**, and performs a full batch conversion of all 35 trajectories, transforming raw HDF5 data into a structured CSV format compatible with modern navigation filters and observers.

### **Key Functions**

*   **Automated Batch Extraction**: Processes the entire seen_subjects_test_set.zip in one pass.
*   **Sensor Synchronization**: Exports IMU, Magnetometer, and Visual Odometry (VO) data.
*   **Ground Truth Standardization**: Converts Google Tango Pose data into a standardized `vo.csv` (Visual Odometry) format.
*   **Metadata Management**: Preserves original `info.json` as `metadata.json` and generates `trajectory_info.json` for each sequence.
*   **Cross-Platform Paths**: Uses pathlib for compatibility across Windows, Linux, and macOS.

### **Dataset Reference**

This utility is designed for the **RoNIN: Robust IMU Double Integration** dataset.  
Source: [https://ronin.cs.sfu.ca/](https://ronin.cs.sfu.ca/)  
*Note: The dataset provides pre-interpolated ground truth data aligned to the 200Hz IMU sampling rate.*

---

# Cell 1: Requirement

In [1]:
# Install necessary libraries for HDF5 processing and data management
!pip install h5py pandas tqdm

# Cell 2: Environment Setup

In [2]:
import os
import json
import shutil
import zipfile
import h5py
import time
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
from tqdm.auto import tqdm # For visual progress tracking

# Suppress warnings for clean output
import warnings
warnings.filterwarnings('ignore')

print("✓ Environment initialized.")

✓ Environment initialized.


# Cell 3: Global Configuration

In [3]:
# Path to the source ZIP file (Update this to your local path)
ZIP_FILE_PATH = Path(r'D:\test\seen_subjects_test_set.zip')

# Base directory for standardized output
OUTPUT_BASE_DIR = Path(r'D:\test\RoNIN_Standardized_Dataset')

# Temporary directory for ZIP extraction (will be cleaned up automatically)
TEMP_EXTRACT_DIR = Path(r'D:\test\RoNIN_temp_extraction')

# Ensure directories exist
OUTPUT_BASE_DIR.mkdir(parents=True, exist_ok=True)
TEMP_EXTRACT_DIR.mkdir(parents=True, exist_ok=True)

print(f"📁 Source ZIP: {ZIP_FILE_PATH}")
print(f"📁 Output Folder: {OUTPUT_BASE_DIR}")

📁 Source ZIP: D:\test\seen_subjects_test_set.zip
📁 Output Folder: D:\test\RoNIN_Standardized_Dataset


# Cell 4: RoNIN Batch Processor Class

In [4]:
class RoNINBatchProcessor:
    """
    Handles extraction, conversion, and cleanup of the RoNIN dataset.
    Fixed: Auto-detects trajectory IDs and pretty-prints JSON metadata.
    """
    def __init__(self, zip_path, output_dir, temp_dir):
        self.zip_path = Path(zip_path)
        self.output_dir = Path(output_dir)
        self.temp_dir = Path(temp_dir)
        self.csv_root = self.output_dir / "csv_data"
        self.csv_root.mkdir(parents=True, exist_ok=True)

    def extract_dataset(self):
        """Unpacks the ZIP file to the temporary directory."""
        print(f"📦 Extracting {self.zip_path.name}... (This may take a few minutes)")
        if not self.zip_path.exists():
            raise FileNotFoundError(f"Source ZIP not found at {self.zip_path}")
            
        with zipfile.ZipFile(self.zip_path, 'r') as zip_ref:
            zip_ref.extractall(self.temp_dir)
        
        # Locate all trajectory folders containing data.hdf5
        # This accurately identifies the subfolder name as the Trajectory ID
        traj_dirs = sorted([p.parent for p in self.temp_dir.rglob('data.hdf5')])
        print(f"✓ Found {len(traj_dirs)} trajectories.")
        return traj_dirs

    def convert_to_csv(self, traj_dir, traj_id):
        """Converts a single HDF5 trajectory to standardized CSVs with tidy JSON."""
        try:
            hdf5_path = traj_dir / "data.hdf5"
            info_path = traj_dir / "info.json"

            # 1. Read HDF5 Data
            with h5py.File(hdf5_path, 'r') as h5:
                sync = h5['synced']
                t = sync['time'][:]
                acc, gyro, mag = sync['acce'][:], sync['gyro'][:], sync['magnet'][:]
                
                pose = h5['pose']
                vo_pos = pose['tango_pos'][:]

            # 2. Create Target Directory (Using the detected traj_id)
            target_path = self.csv_root / traj_id
            target_path.mkdir(parents=True, exist_ok=True)

            # 3. Export Sensors (200Hz)
            pd.DataFrame({
                'time': t, 'ax': acc[:,0], 'ay': acc[:,1], 'az': acc[:,2],
                'wx': gyro[:,0], 'wy': gyro[:,1], 'wz': gyro[:,2]
            }).to_csv(target_path / 'imu.csv', index=False)

            pd.DataFrame({
                'time': t, 'mx': mag[:,0], 'my': mag[:,1], 'mz': mag[:,2]
            }).to_csv(target_path / 'mag.csv', index=False)

            pd.DataFrame({
                'time': t, 'X': vo_pos[:,0], 'Y': vo_pos[:,1], 'Z': vo_pos[:,2]
            }).to_csv(target_path / 'vo.csv', index=False)

            # 4. Process Metadata (Tidy JSON formatting)
            if info_path.exists():
                with open(info_path, 'r') as f_in:
                    metadata_content = json.load(f_in)
                with open(target_path / 'metadata.json', 'w') as f_out:
                    json.dump(metadata_content, f_out, indent=4)

            # 5. Generate Trajectory Summary
            summary = {
                'trajectory_id': traj_id,
                'imu_samples': len(t),
                'duration_sec': round(float(t[-1] - t[0]), 3),
                'processed_at': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            }
            with open(target_path / 'trajectory_info.json', 'w') as f:
                json.dump(summary, f, indent=4)

            return True
        except Exception as e:
            print(f"Error processing {traj_id}: {e}")
            return False

    def run_pipeline(self):
        """Extract -> Convert -> Cleanup."""
        start_time = time.time()
        
        # Step 1: Extraction
        traj_dirs = self.extract_dataset()
        
        # Step 2: Batch Processing
        success_count = 0
        for traj_path in tqdm(traj_dirs, desc="Standardizing Dataset"):
            # The folder name (e.g. a000_11) is the ID
            traj_id = traj_path.name
            
            if self.convert_to_csv(traj_path, traj_id):
                success_count += 1

        # Step 3: Cleanup
        print(f"🧹 Cleaning up temporary files at {self.temp_dir}...")
        shutil.rmtree(self.temp_dir, ignore_errors=True)
        
        total_time = time.time() - start_time
        print(f"\n{'='*50}")
        print(f"COMPLETION SUMMARY")
        print(f"{'='*50}")
        print(f"Total Trajectories Found: {len(traj_dirs)}")
        print(f"Successfully Processed:   {success_count}")
        print(f"Total Processing Time:    {total_time/60:.2f} minutes")
        print(f"Output Location: {self.csv_root.absolute()}")

# Cell 5: Execution

In [5]:
# Instantiate the processor
processor = RoNINBatchProcessor(
    zip_path=ZIP_FILE_PATH,
    output_dir=OUTPUT_BASE_DIR,
    temp_dir=TEMP_EXTRACT_DIR
)

# Run the full batch processing pipeline
processor.run_pipeline()

📦 Extracting seen_subjects_test_set.zip... (This may take a few minutes)
✓ Found 35 trajectories.


Standardizing Dataset:   0%|          | 0/35 [00:00<?, ?it/s]

🧹 Cleaning up temporary files at D:\test\RoNIN_temp_extraction...

COMPLETION SUMMARY
Total Trajectories Found: 35
Successfully Processed:   35
Total Processing Time:    1.18 minutes
Output Location: D:\test\RoNIN_Standardized_Dataset\csv_data


# Cell 6: Dataset Verification

In [9]:
# List processed folders to verify trajectory IDs
csv_data_path = OUTPUT_BASE_DIR / "csv_data"
if csv_data_path.exists():
    processed_trajectories = sorted([f.name for f in csv_data_path.iterdir() if f.is_dir()])
    print(f"Total Processed Folders: {len(processed_trajectories)}")
    print(f"First 5 Trajectory IDs: {processed_trajectories[:5]}")
    
    # Check the tidy JSON of the first trajectory
    sample_json = csv_data_path / processed_trajectories[0] / 'trajectory_info.json'
    if sample_json.exists():
        print(f"\nContent of {processed_trajectories[0]}/trajectory_info.json:")
        with open(sample_json, 'r') as f:
            print(f.read())

✓ Success! CSV files are saved in the 'csv_data' directory.
